In [17]:
# ! conda install scikit-learn -y

In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem
from mordred import Calculator, descriptors

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from mordred import Calculator, descriptors

In [45]:
dataset = pd.read_csv('mTOR_dataset.csv', lineterminator='\n')

# Extract features and target
# Split the train and test set

non_numeric_metadata_columns = ["molecule_chembl_id", "class", "canonical_smiles"]

dataset = dataset.drop(non_numeric_metadata_columns, axis=1)

X = dataset.drop('pIC50', axis=1)
Y = dataset.pIC50

In [46]:
#############################
# Test on Specific Molecule #
#############################
# Production / Deployment  #
############################


molecules_of_interest = {
    'Name': [
        'Arginine', 
        'Histidine', 
        'Isoleucine', 
        'Leucine', 
        'L-Leucine', 
        'Lysine', 
        'Methionine', 
        'Phenylalanine', 
        'Threonine', 
        'Tryptophan', 
        'L-Tryptophan', 
        'Valine', 
        'Alanine', 
        'Asparagine', 
        'Aspartate', 
        'Glutamate', 
        'Glycine', 
        'Serine', 
        'Tyrosine', 
        'Cysteine', 
        'Glutamine', 
        'Hydroxyproline', 
        'Proline', 
        'Taurine'
            ],
    'canonical_smiles': [
        'CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)CC(OC(=O)C3CCCCN3C(=O)C(=O)C1(O2)O)C(C)CC4CCC(C(C4)OC)O)C)C)O)OC)C)C)C)OC', 
        'C1=C(NC=N1)CC(C(=O)O)N', 'CCC(C)C(C(=O)O)N', 'CC(C)CC(C(=O)O)N', 'NC(C(=O)O)CC(C)C', 'C(CCN)CC(C(=O)O)N', 'CSCCC(C(=O)O)N', 'C1=CC=C(C=C1)CC(C(=O)O)N', 'CC(C(C(=O)O)N)O', 'C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N', 'OC(=O)C(Cc1c[nH]c2c1cccc2)N', 'CC(C)C(C(=O)O)N', 'CC(C(=O)O)N', 'C(C(C(=O)O)N)C(=O)N', 'C(C(C(=O)[O-])[NH3+])C(=O)[O-]', 'C(CC(=O)[O-])C(C(=O)[O-])[NH3+]', 'C(C(=O)O)N', 'C(C(C(=O)O)N)O', 'C1=CC(=CC=C1CC(C(=O)O)N)O', 'C(C(C(=O)O)N)S', 'C(CC(=O)N)C(C(=O)O)N', 'C1C(CNC1C(=O)O)O', 'C1CC(NC1)C(=O)O', 'C(CS(=O)(=O)O)N'
    ]
}

target_df = pd.DataFrame(molecules_of_interest)

########################
# Lipinski Descriptors #
########################

# Function to calculate lipinski descriptors

def lipinski(smiles, verbose=False):

    moldata = []
    
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

df_lipinski = lipinski(target_df.canonical_smiles)
target_df = pd.concat([target_df, df_lipinski], axis=1)


###########################
# Adding more Descriptors #
###########################

# Convert smiles to molecules

calc = Calculator(descriptors, ignore_3D=True)
mols = [Chem.MolFromSmiles(smi) for smi in target_df.canonical_smiles.tolist()]

# Calculate Descriptors for enzymes

descriptors_df = calc.pandas(mols)

target_df = pd.concat([descriptors_df, target_df], axis=1)

#################
# Preprocessing #
#################

target_df = target_df.apply(pd.to_numeric, errors='coerce').dropna(axis=1)

# Imputing Missing Values - change to regression imputing (more accurate)
# # fit regression model using Bayesian Ridge
# imputer = IterativeImputer(estimator=BayesianRidge())

# # impute missing values
# imputed_data = imputer.fit_transform(features)

target_df = target_df.fillna(X.mean())


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 11.50it/s]


/home/vitsuji/anaconda3/envs/BioActivity/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [47]:

# Keep only the common columns

common_columns = list(set(X.columns.tolist()) & set(target_df.columns.tolist()))

target_df = target_df[common_columns]
target_df = target_df.loc[:,~target_df.columns.duplicated()].copy()

X = X[common_columns]
X = X.loc[:,~X.columns.duplicated()].copy()


In [51]:
##################
# Training Model #
##################

# Split Dataset

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

dtr = DecisionTreeRegressor(max_depth=3)
abr = AdaBoostRegressor(estimator=dtr, n_estimators=50, learning_rate=0.1, loss='linear', random_state=42)

model_result = abr.fit(X_train, Y_train)

print('Model Score Train', model_result.score(X_train, Y_train))
print('Model Score Test', model_result.score(X_test, Y_test))

y_pred = abr.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
print("Mean Squared Error:", mse)

Model Score Train 0.9186675729352355
Model Score Test 0.8642431769623212
Mean Squared Error: 0.4242921672147465


In [54]:
# Predict the BioActivity of Target Molecules

outputs = abr.predict(target_df)

print(outputs)

[8.67280922 4.98285695 4.9722159  5.08011313 5.08011313 5.08011313
 5.11804049 5.10574426 4.97669831 5.11445077 5.11445077 4.98285695
 4.98285695 4.9722159  4.98285695 4.98285695 4.9722159  4.9722159
 5.08011313 5.03884876 4.98285695 4.98285695 5.08011313 5.07718603]
